<a href="https://colab.research.google.com/github/BrenyQT/CS4287-Atari/blob/main/CS4287_Prj2_21315205_ID2_ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Brendan Quinn: 21315205
Kevin Hough:
Sophie Quinn:

2. The code excecutes without error

3. Resources:

# 1. **Why Reinforcement Learning is the machine learning paradigm of choice for this task:**

Reinforcement Learning is suitable for solving Atari Pong because:

 **Sequential Decision Making:** Pong involves moving a paddle up and down to hit aball back toward an opponent. This means a decision needs to be made at each time step. The agents goal is to maximise the games score by leatrning an optimal sequence of movements. Reinforcement Learning allows for solving problems where the current decision impacts future rewards.

**Dynamic and Interactive Learning Environment:** Pong is an interactive game where the balls position and direction are directly influenced bt the agents actions and the opponents behaviour. Reinforcemeent Learning adapts to the eveolving enviroment by learning through interaction.

**Reward Driven Learning:** The reward signal in pong is straightforward. A postive reward is given for scoring a point. A negative reward is given for coding a point. Reinforcement Learning is designed to optimize cumulative rewards. This makes it well suited for tasks with such clearly defined goals.

# 2. **The Gym Environment:**
 The Gym Enviroment was developled by Open AI in 2016. The Gym Enviroment is a widely used API for developing and eveluating Reinforcement Learning algorithms. It provides a standerdized interface and a diverse set of enviroments, including classic control tasks, board games and Atari games. This standeardization provides a benchmarking enviroment where several Reinforcemnt Learning algorithms can be tested.

**Standerdized Interface:** Gym provides an API for interacting with various enviroments. This median of stardardization allows researchers to apply their Reinforcement Learning algorithms accross different tasks without significant change.

**Predefined Enviroments:** Gym includes a collection of predefined enviroments such as **'Pong-v0'**. These enviroments come with a predefined state and action spaces, reward structures and dynamics. This allows researchers to focus on algorithm developments rather than enviroment creation.

**Visualization Tools:** Gym provides rendering capbilities to visualize the agents interaction with the enviroment. This aid the debugging process.

By utilizing the Gym Enviroment we can leverage  a standardized platform to develop, test and benchmark our Reinforcement Learing Algorithm effectively.


# 3. **Implementation:**


**A. Capture and pre-processing of the data (3 marks).**

**B. The network structure (4 marks).**


**C. A very clear and extremely detailed discussion on the code where the Q learning update applied to the weights (3 marks).**

For example: what is the update, how is the error
calculated?<br><br>

**D. Independently researched concepts such as random seed initialisation, the impact of regularisers on scores, and techniques to counter catastrophic forgetting and maximisation bias (5 marks).**

An attempt must be made to code and evaluate the efficacy of one of these
concepts. Coding fragments and/or diagrams should be included to illustrate the concepts under
discussion

An attempt must be made to code and evaluate the efficacy of one of these
concepts. Coding fragments and/or diagrams should be included to illustrate the concepts under discussion.

# 4. **Plots:**

# 5. **Videos and plots with short accompanying explanations of the information conveyed:**

# 6. **Evaluation of the results:**

A. How does one evaluate the performance of the RL agent?

B. Are the metrics that we have seen to date relevant?

# 7. **References:**

In [1]:
# Install necessary libraries
!pip install tensorflow opencv-python matplotlib numpy

import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import random
from collections import deque
import cv2
import matplotlib.pyplot as plt


In [ ]:
# Preprocessing function
def preprocess_frame(frame):
    # Convert to grayscale and resize to (84, 84)]
    if frame.ndim == 3 and frame.shape[2] == 3:
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    frame = cv2.resize(frame, (84, 84))
    return frame / 255.0  # Normalize pixel values

# Stack frames for temporal information
def stack_frames(stack, new_frame, stack_size=4):
    if stack is None:
        # Initialize stack with repeated frames
        stack = [new_frame for _ in range(stack_size)]
    else:
        # Remove oldest frame and add new frame
        stack = list(stack)
        stack.pop(0)
        stack.append(new_frame)
    return np.stack(stack, axis=-1)


In [ ]:
def build_dqn(input_shape, action_space):
    model = models.Sequential([
        layers.Conv2D(32, (8, 8), strides=4, activation='relu', input_shape=input_shape),
        layers.Conv2D(64, (4, 4), strides=2, activation='relu'),
        layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(action_space, activation='linear')
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='mse')
    return model


In [ ]:
class DQNAgent:
    def __init__(self, action_space):
        self.action_space = action_space
        self.memory = deque(maxlen=20000)
        self.gamma = 0.99  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.batch_size = 64
        self.model = build_dqn((84, 84, 4), action_space)
        self.target_model = build_dqn((84, 84, 4), action_space)
        self.update_target_model()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in batch:
            target = self.model.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * np.amax(t[0])

            self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
!pip install gym[atari] autorom[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=7d99645f021e86937d22f9997365e326fe8dd3707a8340d12dff3f2fb9582f30
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
!pip install gymnasium[atari] autorom[accept-rom-licesnse]
!AutoROM --accept-license
!pip install ale-py

import ale_py
import gymnasium as gym

gym.register_envs(ale_py)

env = gym.make('ALE/Pong-v5')

agent = DQNAgent(action_space=env.action_space.n)
episodes = 500
stack_size = 4

rewards = []  # Initialize rewards list

for episode in range(episodes):
    state, info = env.reset()
    state = preprocess_frame(state)  # Use the initial state returned by env.reset()
    state_stack = stack_frames(None, state, stack_size)
    state_stack = np.expand_dims(state_stack, axis=0)

    total_reward = 0
    done = False
    while not done:
        action = agent.act(state_stack)
        next_state, reward, done, truncated, info = env.step(action)
        total_reward += reward

        if done or truncated:
            break

        next_state = preprocess_frame(next_state)
        next_state_stack = stack_frames(state_stack[0], next_state, stack_size)
        next_state_stack = np.expand_dims(next_state_stack, axis=0)

        agent.remember(state_stack, action, reward, next_state_stack, done)
        state_stack = next_state_stack

        agent.replay()

    agent.update_target_model()
    print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")
    rewards.append(total_reward)  # Append reward to list

# Plot rewards over episodes
import matplotlib.pyplot as plt

plt.plot(rewards)
plt.xlabel('Episodes')
plt.ylabel('Total Reward')
plt.title('Performance of DQN on Pong')
plt.show()

# Save model
agent.model.save('pong_dqn.h5')


In [ ]:
# Plot rewards over episodes
plt.plot(rewards)
plt.xlabel('Episodes')
plt.ylabel('Total Reward')
plt.title('Performance of DQN on Pong')
plt.show()

# Save model
agent.model.save('pong_dqn.h5')


NameError: name 'rewards' is not defined